# Introduction
The Nominatim API is a REST API (provided by OpenStreetMap), that returns geographic coordinates (latitude and longitude) from a postal address. If several postal addresses (due to inaccuracy or non-existent street number) corresponding to the request are found, the API returns several coordinates, each time with an `importance` score. The coordinates are given in descending order of importance, so you can select only the first address.

In [1]:
# Execute the code below :

import requests
link = "https://nominatim.openstreetmap.org/?q=54+Via+Pietro+Mascagni,Catania,Italy&format=json"
r = requests.get(link).json()
r

[{'place_id': 99008230,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'way',
  'osm_id': 31183303,
  'boundingbox': ['37.5115262', '37.5119158', '15.0979776', '15.1011722'],
  'lat': '37.5117512',
  'lon': '15.0995581',
  'display_name': 'Via Pietro Mascagni, Picanello-Ognina-Barriera-Canalicchio, Catania, Sicilia, 95127, Italia',
  'class': 'highway',
  'type': 'residential',
  'importance': 0.51},
 {'place_id': 185547267,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'way',
  'osm_id': 409895559,
  'boundingbox': ['37.5110212', '37.5113841', '15.0942789', '15.0969174'],
  'lat': '37.5112366',
  'lon': '15.0957499',
  'display_name': 'Via Pietro Mascagni, Picanello-Ognina-Barriera-Canalicchio, Catania, Sicilia, 95129, Italia',
  'class': 'highway',
  'type': 'residential',
  'importance': 0.51},
 {'place_id': 118977935,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0

In [2]:
# Here we select only the first address (index 0)
print("First address :", r[0])
print("First address longitude :",r[0]['lon'])
print("First address latitude :",r[0]['lat'])

First address : {'place_id': 99008230, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 31183303, 'boundingbox': ['37.5115262', '37.5119158', '15.0979776', '15.1011722'], 'lat': '37.5117512', 'lon': '15.0995581', 'display_name': 'Via Pietro Mascagni, Picanello-Ognina-Barriera-Canalicchio, Catania, Sicilia, 95127, Italia', 'class': 'highway', 'type': 'residential', 'importance': 0.51}
First address longitude : 15.0995581
First address latitude : 37.5117512


In [3]:
# For ease of use, you can add a limit on the number of items returned:

link = "https://nominatim.openstreetmap.org/?q=54+Via+Pietro+Mascagni,Catania,Italy&format=json"
r = requests.get(link).json()
print("WITHOUT limit, how many coordinates does this address return?",len(r))

link = "https://nominatim.openstreetmap.org/?q=54+Via+Pietro+Mascagni,Catania,Italy&format=json&limit=1"
r = requests.get(link).json()
print("WITH limit, how many coordinates does this address return?",len(r))

WITHOUT limit, how many coordinates does this address return? 10
WITH limit, how many coordinates does this address return? 1


## How to create your API query
It's up to you to modify the string to create the right request URL

In [5]:
# We observe that the query consists of a fixed part, followed by the address to be searched for.
# An URL cannot contain a " " space character, 
# and special characters or accents should be avoided if possible

link_main = 'https://nominatim.openstreetmap.org/?q='
address = '54 Via Pietro Mascagni, Catania, Italy'
link_end = '&format=json&limit=1'

link = link_main + address.replace(", ",",").replace(" ","+") # It's your turn here. Your goal is to get a link in the right format

print(link + link_end)

https://nominatim.openstreetmap.org/?q=54+Via+Pietro+Mascagni,Catania,Italy&format=json&limit=1


In [6]:
# Create a function here that turns a postal address into a request URL for the Nominatim API, 
# then makes the request and returns the coordinates :

def API_address(postal_address):
    link_main = 'https://nominatim.openstreetmap.org/?q='
    link_end = '&format=json&limit=1'

    link = link_main + postal_address.replace(", ",",").replace(" ","+")

    return (link + link_end)
  


In [7]:
# Test it here:
API_address(address)

'https://nominatim.openstreetmap.org/?q=54+Via+Pietro+Mascagni,Catania,Italy&format=json&limit=1'

# DataViz
Latitude & Longitude can be used on visualization tools, whether they are BI tools (PowerBI, Table), or Python DataViz libraries such as Plotly or Folium.

Here we will display a map with Folium.


In [ ]:
# The syntax of Folium is very simple, you start by creating a map centered on a point.
# You can change the default zoom level with the argument "zoom_start".

import folium

# We define a location as a list with 2 values : latitude and longitude.
point = [float(r[0]['lat']), float(r[0]['lon'])]

# We center the map on the location 
m = folium.Map(location=point,zoom_start=7)

# We display the map
m

In [ ]:
# Then you can add landmarks and put a clickable comment
m = folium.Map(location=point, )
folium.Marker(
    location=point,
    popup='a good restaurant'
    ).add_to(m)
m

# Challenge
Here is a DataFrame with restaurants in Catania, Sicily, and their respective addresses. Here is your mission:
- Create a new column "coordinates", which will store the coordinates corresponding to each address (you can use the function you created previously).
- Display a map with the 4 restaurant markers. Be careful, the restaurants are very close, remember to set the default zoom level so that it is clearly legible. You can center the map on the first restaurant. And display the name of the restaurant in the tooltip popup.

In [70]:
import numpy as np
import pandas as pd
restaurants = pd.DataFrame([["Gelateria Zio Pietro dal 1964", "Via Porta di Ferro, 47, 95131 Catania CT"],
                            ["A Casa d'Amici","Via Fischetti, 14, 95131 Catania CT"],
                            ["La Bitta", "Via Acquicella Porto, 95121 Catania CT"],
                            ["Steak House", "Via Porta di Ferro, 8, 95100 Catania CT"]],
                           columns = ["name", "address"])

# This is done in loop as I didn't find better way
#   But there might be a viable .apply(func) solution
#   The func could be applied on the restaurant["address"]
#   and for each address value it would issue an API request
#   and create two new entries into new columns

# We're looping over index as we need to work with index later

# We need to create empty series with dtype=object,
#   so we could store "list" of latitude and longitude into it
s = pd.Series(dtype=object)

for index in restaurants["address"].index:

    # Getting address using index
    address = restaurants["address"][index]

    # Gettin response from the address lookup
    r = requests.get(API_address(address)).json()

    # Response is a list of dicts, so we run a loop through items
    for item in r:
        # insert values on specific place into the dataframe
        # this is where we needed the index mostly
        #restaurants.at[index, "lat"] = float(item["lat"])
        #restaurants.at[index, "lon"] = float(item["lon"])
        #
        #restaurants.at[index, "coordinates"] = [float(item["lat"]), float(item["lon"])]
        s.at[index] = [float(item["lat"]), float(item["lon"])]

# Now insert the new Series into the dataframe
restaurants["coordinates"] = s

In [110]:
# Filtering out only items that were found
found_items = restaurants[~restaurants["coordinates"].isna()].reset_index()

# Setting the initial location of the map (first item)
point = found_items.at[0, "coordinates"]

In [114]:
# We center the map on the location 
m = folium.Map(location=point, zoom_start=14)

for index in restaurants[~restaurants["coordinates"].isna()].index:
    #print(f"coordinates: {restaurants.at[index, 'coordinates']}")
    #print(f"popup: {restaurants.at[index, 'name']}")
    folium.Marker(
        location=restaurants.at[index, "coordinates"],
        popup=restaurants.at[index, "name"]
        ).add_to(m)

# We display the map
m

# Remarks on the Nominatim API
As indicated in the quest, there are many resources for geocoding. Most of them are available by registration, and some require a fee.

The Nominatim API is free and without registration. The disadvantage is that it is relatively slow. If you need to use it in the future, remember to store the results so you don't have to run it several times.

For your knowledge, there is also :
- the **reverse** address API, which allows you to find the nearest postal address using geographic coordinates.
- the API from a **CSV file** if you have a lot of addresses to geocode
- the API **GeoJSON** which allows to obtain a geoJSON format of locations to make choropleth maps


All the [documentation is available here](https://nominatim.org/release-docs/develop/api/Search/).